## Download repo

In [1]:
import os
import shutil
import zipfile
import urllib.request

def download_repo(url, save_to):
    zip_filename = save_to + '.zip'
    urllib.request.urlretrieve(url, zip_filename)
    
    if os.path.exists(save_to):
        shutil.rmtree(save_to)
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall('.')
    del zip_ref
    assert os.path.exists(save_to)

In [2]:
REPO_PATH = 'LinearizedNNs-master'

# download_repo(url='https://github.com/maxkvant/LinearizedNNs/archive/master.zip', save_to=REPO_PATH)

## Imports

In [3]:
import sys
sys.path.append(f"{REPO_PATH}/src")

In [4]:
import time
import numpy as np
import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
from torchvision.datasets import FashionMNIST

from estimator import Estimator
from pytorch_impl.estimators import SgdEstimator
from pytorch_impl.nns import Myrtle5, Myrtle7, Myrtle10
from pytorch_impl import ClassifierTraining
from pytorch_impl.matrix_exp import matrix_exp, compute_exp_term
from pytorch_impl.nns.utils import to_one_hot, print_sizes
from from_neural_kernels import to_zca, CustomTensorDataset, get_cifar_zca

In [5]:
device = torch.device('cuda:0') if (torch.cuda.is_available()) else torch.device('cpu')
device

device(type='cuda', index=0)

## Cifar 10

In [17]:
class GpEstimator(Estimator):
    def __init__(self, models, n_classes, learning_rate, x_example, device):
        super(GpEstimator, self).__init__()
        self.models    = models
        self.lr        = learning_rate
        self.n_classes = n_classes
        self.device    = device
        
        n = len(models)
        X = torch.stack([x_example]).to(device)
        
        model = models[0].to(device)
        readout_size = model.readout(X).size()[1]
    
        # TODO: Assert that models have the same readout size
        
        self.w      = torch.zeros([n, readout_size, n_classes]).to(device)
        self.w_size = n * readout_size 
        
    def get_w_update(self, X, right_vector):
        with torch.no_grad():
            assert len(X) == len(right_vector)

            X            = X.to(self.device)
            right_vector = right_vector.to(self.device)

            n = len(X)
            w_updates = []
            
            for model in models:
                features = self.to_model_features(X, model)
                update = torch.matmul(features.T, right_vector)
                w_updates.append(update)
            return torch.stack(w_updates)
                                  
    def to_model_features(self, X, model):
        with torch.no_grad():
            model = model.to(device)
            return model.readout(X) * (1. / np.sqrt(self.w_size))
        
    def calc_kernel(self, X):
        with torch.no_grad():
            X = X.to(device)
            
            res = torch.zeros([len(X), len(X)]).to(device)
            for model in models:
                features_x = self.to_model_features(X, model)
                
                res += torch.matmul(features_x, features_x.T)
            return res
        
    def calc_kernels(self, X_train, X_test):
        with torch.no_grad():
            X_train = X_train.to(device)
            X_test  = X_test.to(device)
            
            res_train = torch.zeros([len(X_train), len(X_train)]).to(device)
            res_test  = torch.zeros([len(X_test),  len(X_train)]).to(device)
            for model in models:
                features_train = self.to_model_features(X_train, model)
                features_test  = self.to_model_features(X_test,  model)
                
                res_train += torch.matmul(features_train, features_train.T)
                res_test  += torch.matmul(features_test,  features_train.T)
            return res_train, res_test
            
    def predict(self, X):
        X = X.to(self.device)
        with torch.no_grad():
            n = len(X)
            res = torch.zeros([n, self.n_classes]).to(device)

            for model, w in zip(models, self.w):
                features = self.to_model_features(X, model)
                res += torch.matmul(features, w)
            return res

In [7]:
def calc_right_vector(kernel, y, learning_rate=1e4, reg_param=0):
    with torch.no_grad():
        y      = y.to(device)
        kernel = kernel.to(device)
        
        n      = len(kernel)
        reg = torch.eye(n).to(device) * reg_param
        
        exp_term = - learning_rate * compute_exp_term(- learning_rate * (kernel + reg), device)
        right_vector = torch.matmul(exp_term, - y)
        return right_vector

In [14]:
%%time

X_train, labels_train, X_test, labels_test = get_cifar_zca()

N_train = 12800
N_test  = 1000

X_train      = torch.tensor(X_train[:N_train]).float()
labels_train = torch.tensor(labels_train[:N_train], dtype=torch.long)

X_test       = torch.tensor(X_test[:N_test]).float()
labels_test  = torch.tensor(labels_test[:N_test],  dtype=torch.long)

num_classes = 10

y_train = to_one_hot(labels_train, num_classes).to(device)
y_test  = to_one_hot(labels_test,  num_classes).to(device)

Files already downloaded and verified
Files already downloaded and verified
CPU times: user 3min 49s, sys: 1min 4s, total: 4min 54s
Wall time: 44.3 s


In [15]:
cifar_train = CustomTensorDataset(torch.tensor(X_train), torch.tensor(labels_train, dtype=torch.long), transform='all')
train_loader = torch.utils.data.DataLoader(cifar_train, batch_size=1280)

/home/maxkvant/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


## Log Reg

In [32]:
n_models = 5000

models = [Myrtle5(num_filters=32) for _ in range(n_models)]

learning_rate = 0.2
estimator = GpEstimator(models, num_classes, learning_rate, X_train[0], device)

n_iter = 20

for iter_ in range(n_iter):
    print(f"iter {iter_} ======================= ")
    
    for batch_i, (X, labels) in enumerate(train_loader):
        batch_start = time.time()
        
        X      = X.to(device)
        labels = labels.to(device)
        
        y_pred = estimator.predict(X)
        train_acc = (y_pred.argmax(dim=1) == labels).float().mean().item()
        print(f"train_acc {train_acc}")
        
        y_pred.requires_grad = True

        loss = nn.CrossEntropyLoss()(y_pred, labels)
        loss.backward()
        grad = y_pred.grad
        
        estimator.w += -learning_rate * estimator.get_w_update(X, grad)
        
        print(f"batch_{batch_i} took {time.time() - batch_start:0}s")
        print()
        
    y_pred = estimator.predict(X_test)
    test_acc = (y_pred.argmax(dim=1) == labels_test.to(device)).float().mean().item()
    print(f"iter {iter_} done, test_acc = {test_acc}")

iter 0 ======================= 
train_acc 0.09921874850988388
batch_0 took 57.80506658554077s

train_acc 0.09140624850988388
batch_1 took 55.33869671821594s

train_acc 0.1328125
batch_2 took 55.32820272445679s

train_acc 0.09296875447034836
batch_3 took 55.33759117126465s

train_acc 0.09375
batch_4 took 55.329872846603394s

train_acc 0.1171875
batch_5 took 55.33920168876648s

train_acc 0.09765625
batch_6 took 55.32954454421997s

train_acc 0.17734375596046448
batch_7 took 55.33659553527832s

train_acc 0.09375
batch_8 took 55.32691836357117s

train_acc 0.17499999701976776
batch_9 took 55.34224081039429s

iter 0 done, test_acc = 0.1120000034570694
iter 1 ======================= 
train_acc 0.11015625298023224
batch_0 took 55.32840847969055s



KeyboardInterrupt: 

## Matrix exp

In [ ]:
%%time

X_train, labels_train, X_test, labels_test = get_cifar_zca()

N = 1280

X_train      = torch.tensor(X_train[:N]).float()
X_test       = torch.tensor(X_test[:N]).float()
labels_train = torch.tensor(labels_train[:N], dtype=torch.long)
labels_test  = torch.tensor(labels_test[:N],  dtype=torch.long)

num_classes = 10

y_train = to_one_hot(labels_train, num_classes).to(device)
y_test  = to_one_hot(labels_test,  num_classes).to(device)

In [9]:
n_models = 5000

models = [Myrtle5(num_filters=32) for _ in range(n_models)]

estimator = GpEstimator(models, num_classes, 1e4, X_train[0], device)

In [10]:
%%time
train_kernel, test_kernel = estimator.calc_kernels(X_train, X_test)

right_vector = calc_right_vector(train_kernel, y_train, learning_rate=1e5, reg_param=0)
y_pred = torch.matmul(test_kernel, right_vector).argmax(dim=1)

test_acc = (y_pred.cpu() == labels_test).float().mean()
print(test_acc)
print()

tensor(0.6102)

CPU times: user 1min 16s, sys: 25.5 s, total: 1min 42s
Wall time: 1min 42s


In [11]:
%%time
w_update = estimator.get_w_update(X_train, right_vector)

print(w_update.size())
print()

torch.Size([5000, 32, 10])

CPU times: user 34.7 s, sys: 14.7 s, total: 49.4 s
Wall time: 49.4 s


In [12]:
estimator.w = w_update

In [13]:
%%time

y_pred = estimator.predict(X_test).argmax(dim=1)

test_acc = (y_pred.cpu() == labels_test).float().mean()
print(test_acc)
print()

tensor(0.6102)

CPU times: user 35.2 s, sys: 14.9 s, total: 50.1 s
Wall time: 50 s
